In [15]:
from urllib.request import Request, urlopen
from datetime import *
from bs4 import BeautifulSoup as bs
import openpyxl
import pygame
import re
import time

In [16]:
url = 'https://www.wista.net/en/wistabook/wistabook/wista_lang,33'

request = Request(url)
json = urlopen(request).read()
array = str(json)
soup = bs(array)
prettyHTML = soup.prettify()
prettyHTML = prettyHTML.split('\n')

/usr/lib/python3/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/usr/lib/python3/dist-packages/bs4/builder/_lxml.py:240: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)


In [17]:
urls = []
regexURL = 'href="(.*?)"'
for i in range(977, len(prettyHTML)):
    if 'show_profile' in prettyHTML[i]:
        matchURL = re.search(regexURL, prettyHTML[i])
        if matchURL:
            urls.append(matchURL.group(1))
            print(matchURL.group(1))

https://www.wista.net/en/wistabook/wistabook/show_profile,88/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,105/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,106/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,150/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,142/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,280/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,284/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,312/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,168/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,379/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,408/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,412/wista_lang,33
https://www.wista.net/en/wistabook/wistabook/show_profile,141/wista_lang,33
https://www.w

In [18]:
dicty = {}
dicty['PROFILE']                = "A"
dicty['COMPANY']                = "B"
dicty['TITLE']                  = "C"
dicty['ADDRESS']                = "D"
dicty['EMAIL']                  = "E"
dicty['WEBSITE']                = "F"
dicty['FUNCTIONALCATEGORIES']   = "G"
dicty['SHIPPINGCATEGORIES']     = "H"
dicty['JOBDESCRIPTION']         = "I"
dicty['DESCRIPTIONOFCOMPANY']   = "J"
dicty['STARTEDINSHIPPING']      = "K"
dicty['OFFICEPHONE']            = 'L'
dicty['MOBILEPHONE']            = 'M'
dicty['FAX']                    = 'N'
dicty['LASTUPDATE']             = "O"

# Create a new excel file
out = openpyxl.Workbook()
# Open the worksheet we want to edit
outsheet = out.create_sheet("wista members")
# if 'sheet' appears randomly we can delete it
rm = out.get_sheet_by_name('Sheet')
out.remove_sheet(rm)
#################
# DO STUFF HERE #
#################
outsheet[dicty['PROFILE']               + '1'].value = "Profile"
outsheet[dicty['COMPANY']               + '1'].value = "Company"
outsheet[dicty['TITLE']                 + '1'].value = "Title"
outsheet[dicty['EMAIL']                 + '1'].value = "Email"
outsheet[dicty['WEBSITE']               + '1'].value = "Website"
outsheet[dicty['FUNCTIONALCATEGORIES']  + '1'].value = "Functioning Categories"
outsheet[dicty['SHIPPINGCATEGORIES']    + '1'].value = "Shipping Categories"
outsheet[dicty['JOBDESCRIPTION']        + '1'].value = "Job Description"
outsheet[dicty['DESCRIPTIONOFCOMPANY']  + '1'].value = "Company Description"
outsheet[dicty['STARTEDINSHIPPING']     + '1'].value = "Started"
outsheet[dicty['OFFICEPHONE']           + '1'].value = 'Office Phone'
outsheet[dicty['MOBILEPHONE']           + '1'].value = 'Mobile Phone'
outsheet[dicty['FAX']                   + '1'].value = 'Fax'
outsheet[dicty['LASTUPDATE']            + '1'].value = "Last Updated"

regexSpaces = '^ *'

row = 2
step = 50
for j in range(0, len(urls)):
    if row % step == 0:
        print('TAKING A BREAK TO SAVE')
        out.save("wista.xlsx")
    print(urls[j])
    request = Request(urls[j])
    json = urlopen(request).read()
    array = str(json)
    soup = bs(array)
    prettyHTML = soup.prettify()
    prettyHTML = prettyHTML.split('\n')
    
    info = []
    for i in range(630, len(prettyHTML)):
        line = prettyHTML[i]
        line = re.sub(regexSpaces, '', line)
        if '<' not in line and '\\n' not in line and len(info) < 23:
            info.append(line)
            #print(line)
    
    if 'Error' in info[0]:
        continue
    
    current = ''
    newInfo = []
    skip = True
    afterAddress = False
    after = 0
    # take every other line
    # if the line ends with a comma we add it to previous line, otherwise we start over
    # skip every other line unless it has a comma in it, then dont skip the next one
    for i in range(0, len(info)):
        current = current + ' ' + info[i]
        if ',' not in info[i][-1]:
            # add to excel sheet and reset current
            current = current.replace('&amp;', '&')
            print(current)
            if afterAddress:
                after = after + 1
            if after >= 2:
                after = 0
                afterAddress = False
            if not afterAddress or after > 2:
                newInfo.append(current)
                current = ''
        if 'Address' in info[i]:
            afterAddress = True
    
    for i in range(0, len(newInfo), 2):
        column = newInfo[i].upper().replace(' ', '').replace('-', '')
        print(column, row)
        inf = newInfo[i + 1]
        outsheet[dicty[column] + str(row)].value = inf
        if column == 'LASTUPDATE' or newInfo[i + 1].upper().replace(' ', '').replace('-', ''):
            break
    row = row + 1
        

# Save the file
out.save("wista.xlsx")

# LMK when the script is done
pygame.init()
pygame.mixer.music.load('/home/andrefisch/python/evan/note.mp3')
pygame.mixer.music.play()
time.sleep(5)
pygame.mixer.music.stop()


https://www.wista.net/en/wistabook/wistabook/show_profile,88/wista_lang,33


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:23: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:24: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
/usr/lib/python3/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/usr/lib/python3/dist-packages/bs4/builder/_lxml.py:240: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)


https://www.wista.net/en/wistabook/wistabook/show_profile,105/wista_lang,33
 Profile
 Jennifer Montgomery
 Last update
 28 June at 01:55 (CET)
 By name of member
 By name of company
 By country
 By functional category
 Argentina
 Australia
 Bahamas
 Belgium
 Bermuda
 Brazil
 Canada
 Cayman Islands
 Cyprus
 Denmark
 Dominican Rep.
 Finland
 France
 Georgia
 Germany
PROFILE 2
https://www.wista.net/en/wistabook/wistabook/show_profile,106/wista_lang,33
 Profile
 Denise Krepp
 Company
 KDRK Consulting
 Title
 President
 Address
 1837 A Street, SE 20003 Washington
 1837 A Street, SE 20003 Washington USA
 E-mail
 kdrkrepp@hotmail.com
 Functional categories
 Advisor / consultancy
 Job description
 Advising businesses and state and local governments on achieving successful outcomes in their interactions with Congress and the U.S. Departments of Homeland Security, Energy, and Transportation.
 Last update
 28 June at 02:56 (CET)
 By name of member
 By name of company
 By country
 By functional ca